# Group 7 Project Proposal
#### Harsh A, Kyle M, Daniel L, Catarina A 

In [10]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
install.packages("kknn")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [23]:
set.seed(1)
water_data <- read_csv("data/heart_failure.csv") |>
    mutate(DEATH_EVENT = as_factor(DEATH_EVENT)) |>
    select(...1, creatinine_phosphokinase, ejection_fraction, platelets, serum_creatinine, serum_sodium, DEATH_EVENT)
water_data <- water_data |> rename_at('...1', ~'age')
water_data

water_split <- initial_split(water_data, prop = 0.75, strata = DEATH_EVENT)

water_training <- training(water_split)
water_testing <- testing(water_split)


water_training

water_training |>
    group_by(DEATH_EVENT) |>
    summarize(count = n()) |>
    head()

New names:
• `` -> `...1`
Rows: 300 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (13): ...1, anaemia, creatinine_phosphokinase, diabetes, ejection_fracti...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,DEATH_EVENT
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
75,582,20,265000,1.9,130,1
55,7861,38,263358,1.1,136,1
65,146,20,162000,1.3,129,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮
45,2413,38,140000,1.4,140,0
50,196,45,395000,1.6,136,0
NA,NA,NA,NA,NA,NA,NA


age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,DEATH_EVENT
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
49,80,30,427000,1.0,138,0
65,52,25,276000,1.3,137,0
70,75,35,223000,2.7,138,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮
60,166,30,62000,1.70,127,1
65,258,25,198000,1.40,129,1
55,1199,20,263358,1.83,134,1


DEATH_EVENT,count
<fct>,<int>
0,153
1,72


In [25]:
options(repr.plot.height = 10, repr.plot.width = 10)

spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
set_engine("kknn")|>
set_mode("classification")

rec <- recipe(DEATH_EVENT~.,data = water_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

fold <- vfold_cv(water_training, v = 5, strata = DEATH_EVENT)

resample_fit <- workflow() |>
       add_recipe(rec) |>
       add_model(spec) |>
       tune_grid(resamples = fold, grid = tibble(neighbors = seq(1,10,by=1))) |>
        collect_metrics() |>
        filter(.metric=="accuracy") |>
        arrange(desc(mean))
resample_fit

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
9,accuracy,binary,0.7239438,5,0.02181999,Preprocessor1_Model09
10,accuracy,binary,0.7239438,5,0.02181999,Preprocessor1_Model10
7,accuracy,binary,0.7111946,5,0.01768300,Preprocessor1_Model07
⋮,⋮,⋮,⋮,⋮,⋮,⋮
4,accuracy,binary,0.6707861,5,0.01873213,Preprocessor1_Model04
1,accuracy,binary,0.6042819,5,0.02196592,Preprocessor1_Model01
2,accuracy,binary,0.6042819,5,0.02196592,Preprocessor1_Model02
